In [12]:
%load_ext autoreload
%autoreload 2

In [1]:
import pymongo
client = pymongo.MongoClient()
db = client.metrics

In [2]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [3]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [5]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-06-18 12:24:35.131462


In [6]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
#regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
cr_tags = vtags

len(unique_words)

1641

In [59]:
[k for k in sorted(tag_freq.keys()) if "->" not in k]

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Anaphor',
 'COMPILED',
 'Causer',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Causer:Anaphor',
 'Causer:other',
 'Causer:rhetorical',
 'Result',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7',
 'Result:Anaphor',
 'Result:other',
 'Result:rhetorical',
 'explicit',
 'other',
 'rhetorical']

In [7]:
sorted(vtags)

['Causer:1->Result:11',
 'Causer:1->Result:13',
 'Causer:1->Result:14',
 'Causer:1->Result:2',
 'Causer:1->Result:3',
 'Causer:1->Result:4',
 'Causer:1->Result:5',
 'Causer:1->Result:50',
 'Causer:1->Result:6',
 'Causer:1->Result:7',
 'Causer:11->Result:11',
 'Causer:11->Result:12',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:11->Result:3',
 'Causer:11->Result:4',
 'Causer:11->Result:50',
 'Causer:11->Result:6',
 'Causer:12->Result:11',
 'Causer:12->Result:13',
 'Causer:12->Result:14',
 'Causer:12->Result:50',
 'Causer:12->Result:5b',
 'Causer:12->Result:7',
 'Causer:13->Result:11',
 'Causer:13->Result:12',
 'Causer:13->Result:14',
 'Causer:13->Result:4',
 'Causer:13->Result:5',
 'Causer:13->Result:50',
 'Causer:13->Result:6',
 'Causer:13->Result:7',
 'Causer:14->Result:50',
 'Causer:14->Result:6',
 'Causer:14->Result:7',
 'Causer:2->Result:1',
 'Causer:2->Result:3',
 'Causer:2->Result:50',
 'Causer:2->Result:6',
 'Causer:3->Result:1',
 'Causer:3->Result:13',
 'Causer:3-

In [8]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    ys_bytag = defaultdict(list)
    ys_bytag_sent = defaultdict(list)
    seq_lens = []

    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set()
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)                
                # remove unwanted tags
                tags = vtags.intersection(tags)
                unique_tags.update(tags)
                # retain all tags for evaluation (not just most common)
                # SKIP the START and END tags
                if word != START and word != END:
                    for t in (vtags - set([EMPTY_TAG])):
                        if t in tags:
                            ys_bytag[t].append(1)
                        else:
                            ys_bytag[t].append(0)

                # encode ys with most common tag only
                if len(tags) > 1:
                    most_common = max(tags, key=lambda t: tag_freq[t])
                    tags = set([most_common])
                if len(tags) == 0:
                    tags.add(EMPTY_TAG)

                one_hot = []
                for t in vtags:
                    if t in tags:
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                y_seq.append(one_hot)
            
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_sent[tag].append(1)
                else:
                    ys_bytag_sent[tag].append(0)
                
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag, ys_bytag_sent, seq_lens

## Feature Extraction

In [41]:
WINDOW_SIZE = 7

config["window_size"] = WINDOW_SIZE
offset = int((config["window_size"] - 1) / 2)
offset

3

In [42]:
from featureextractionfunctions \
    import fact_extract_positional_word_features_stemmed, fact_extract_ngram_features_stemmed,\
    fact_extract_bow_ngram_features, extract_dependency_relation, extract_brown_cluster

unigram_window_stemmed  = fact_extract_positional_word_features_stemmed(offset)
biigram_window_stemmed  = fact_extract_ngram_features_stemmed(offset, 2)
triigram_window_stemmed = fact_extract_ngram_features_stemmed(offset, 3)
unigram_bow_window      = fact_extract_bow_ngram_features(offset, 1)

#TODO - add in full feature set, but keep simple for now
#optimal CB feature set
extractors = [
    unigram_window_stemmed,
    biigram_window_stemmed,
    #triigram_window_stemmed,
    #unigram_bow_window,
    #extract_dependency_relation,
    #extract_brown_cluster
]

In [44]:
from featureextractortransformer import FeatureExtractorTransformer

feature_extractor = FeatureExtractorTransformer(extractors)
essay_feats = feature_extractor.transform(tagged_essays)
len(tagged_essays), len(essay_feats)

In [65]:
l = list(range(0))
l

[]

In [ ]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"
CAUSER_EXPLICIT = "Causer_Explicit"
RESULT_EXPLICIT = "Result_Explicit"
CAUSER_EXPLICIT_RESULT = "Causer_Explicit_Result"

class SearnModel(object):
    Push_Cause = "P_Cause"
    Push_Explicit = "P_Explicit"
    Push_Result = "P_Result"
    Reduce = "Reduce"
    Skip = "Skip"
    
    def __init__(self, feat_extractor, tags, cr_tags, base_learner_fact, beta):
        # init checks
        assert CAUSER in tags, "%s must be in tags" % CAUSER
        assert RESULT in tags, "%s must be in tags" % RESULT
        assert EXPLICIT in tags, "%s must be in tags" % EXPLICIT

        self.feat_extractor = feature_extractor    # feature extractor (for use later)
        self.base_learner_fact = base_learner_fact # SKlearn classifier
        self.tags = set(tags)                      # tags for basic tagging
        self.cr_tags = set(cr_tags)                # causal relation tags
            
        self.actions = set([SearnModel.Push_Cause, SearnModel.Push_Explicit, SearnModel.Push_Result, 
                              SearnModel.Reduce, SearnModel.Skip])
        self.epoch = -1
        self.beta = beta
        self.stack = []
        self.tagging_models = {}
        self.parser_model = None
    
    def get_conditional_feats(self, current_word, prev_words, history):
        feats = {}
        if len(history) > 0:
            #TODO add feats for tags not predicted and actions not taken
            prev_tags = history[-1]
            for t in prev_tags:
                feats["tag-1:" + t] = 1
                feats["tag-1:" + t + " wd:"  + current_word] = 1
                feats["tag-1:" + t + " wd-1:"+ prev_words[-1]] = 1
                
            if len(history) > 1:
                prev_prev_tags = history[-2]
                for t in prev_prev_tags:
                    # tag bigrams
                    for prev_tag in prev_tags:
                        feats["tag-1:" + prev_tag + " tag-2:" + t] = 1
                    feats["tag-2:" + t] = 1
                    feats["tag-2:" + t + " wd:"   + current_word] = 1
                    feats["tag-2:" + t + " wd-2:" + prev_words[-2]] = 1
        
        for i in range(4):
            offset = -(i+1)
            if len(self.stack) < abs(offset):
                break
            s1_key = "s" + str(offset) + ":" + self.stack[offset]
            feats[s1_key] = 1z
            # Stack Bigram feats
            if len(self.stack) >= abs(offset) + 1:
                next_offset = offset-1
                s2_key = "s" + str(next_offset) + ":" + self.stack[next_offset]
                feats[s1_key + " " + s2_key] = 1
        return feats
    
    def __oracle__(self, feats, ground_truth, ground_truth_history):
        # filter to desired tags
        ground_truth_tags = ground_truth.intersection(self.tags)
        pred_tags = set()
        #TODO - flip a coin to decide to use current model's predictions or ground truth
        for tag in self.tags:
            if tag in ground_truth_tags:
                pred_tags.add(tag)
        #Decide Actions
        #TODO what if multiple causer, explicit or results tags assigned to one word?
        if CAUSER in pred_tags:
            pred_tags.add(SearnModel.Push_Cause)
            self.stack.append(CAUSER)
        elif EXPLICIT in pred_tags:
            pred_tags.add(SearnModel.Push_Explicit)
            self.stack.append(EXPLICIT)
        elif RESULT in pred_tags:
            pred_tags.add(SearnModel.Push_Result)
            self.stack.append(RESULT)
        else:
            pred_tags.add(SearnModel.Skip)
            
        # Shift, Reduce or Skip?
        # How to handle end of input (do we try and reduce what's remaining) ?
        if len(ground_truth_history) > 0 and len(self.stack) >=2:
            prev_ground_truth = ground_truth_history[-1]
            for t in cr_tags:
                #TODO: implement parsing actions
                pass
            
    def train(self, essay_feats, epochs):
        #essay_feats = self.feat_extractor.transform(tagged_essays)        
        for i in range(0, epochs):
            self.epoch +=1
            tagging_models = {}
            examples_with_loss = []
            
            for essay_ix, essay in enumerate(essay_feats):
                for sent_ix, taggged_sentence in enumerate(essay.sentences):
                    
                    prediction_history = []
                    ground_truth_history = []
                    previous_words = []
                    self.stack = []
                    for i, (wd) in enumerate(taggged_sentence):
                        
                        # Don't mutate the feat dictionary
                        tagger_feats = dict(wd.features.items())
                        
                        # get all tagger predictions for previous 2 tags
                        cond_feats = self.get_conditional_feats(wd.word, previous_words, prediction_history, ground_truth_history)
                        tagger_feats.update(cond_feats)

                        # add more in depth features for this tag
                        ground_truth = wd.tags
                        oracle_predictions = self.__oracle__(tagger_feats, ground_truth)
                        
                        guess = self.model.predict(tagger_feats)
                        self.model.update(actual, guess, tagger_feats)

                        prev.append(guess)
                        for cls in self.individual_tags:
                            class2predictions[cls].append(  1 if cls in guess  else 0 )
                            class2tags[cls].append(         1 if cls in actual else 0)